In [1]:
%load_ext autoreload
%autoreload 2

# imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('../py_scripts')
import matplotlib.pyplot as plt
from py_scripts.utils_DL import get_net, MNISTDataModule
import pytorch_lightning as pl
from py_scripts.params import get_params, ModelStyles, DataSet, KDecayOptions
import wandb
import torch 
from torch import nn
from pytorch_lightning.loggers import WandbLogger
import numpy as np

# gpu check
!nvidia-smi

/home/trenton/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ImportError: cannot import name 'get_net' from 'py_scripts.utils_DL' (/home/trenton/Foundational-SDM/py_scripts/utils_DL.py)

In [2]:
import torch 
from torch import nn

In [3]:
str(nn.ReLU())

'ReLU()'

In [63]:
torch.nn.EmbeddingBag(100, 10, mode='sum').weight.shape

torch.Size([100, 10])

In [8]:
key_len = 60
n_explicit_keys_per_chunk = 10
nchunks = 3
k = 5
nimplicit_keys = n_explicit_keys_per_chunk**nchunks
assert key_len%nchunks==0, "Need chunks to be evenly divisible."
print("Number of implicit keys", nimplicit_keys)
sub_ks = torch.randn((n_explicit_keys_per_chunk, nchunks,key_len//nchunks))
inputs = torch.randn((128,key_len))
sub_qs = torch.stack(torch.chunk(inputs, nchunks, dim=1),dim=1)
print(sub_ks.shape, sub_qs.shape)

Number of implicit keys 1000
torch.Size([10, 3, 20]) torch.Size([128, 3, 20])


In [9]:
# L2 norm each of the sub components

# TODO: see if the method implemented in the FAIR library is faster. 

sub_ks /= torch.norm(sub_ks, dim=-1, keepdim=True)
sub_qs /= torch.norm(sub_qs, dim=-1, keepdim=True)

In [10]:
full_inputs = sub_qs.flatten(start_dim=1)
full_inputs.shape 

torch.Size([128, 60])

In [41]:
# ground truth test:

for i in range(nchunks-1):
    nreps = n_explicit_keys_per_chunk**(i+1)
    if i==0:
        one = sub_ks[:,i:i+1,:].repeat_interleave(nreps,0)
        two = sub_ks[:,i+1:i+2,:].tile((nreps,1,1))
        full_ks = torch.concat([one, two], dim=-1)
    else: 
        full_ks = full_ks.repeat_interleave(n_explicit_keys_per_chunk**i,0)
        next = sub_ks[:,i+1:i+2,:].tile((nreps,1,1))
        full_ks = torch.concat([full_ks, next], dim=-1)
full_ks = full_ks.squeeze()
    
print("full_ks", full_ks.shape )
# L2 norm each of the sub components

full_ks /= torch.norm(full_ks, dim=-1, keepdim=True)
full_inputs /= torch.norm(full_inputs, dim=-1, keepdim=True)


fsims = torch.matmul(full_ks, full_inputs.T).T
fvals, finds = torch.topk(fsims, k, dim=-1)
target_vals = finds[0]
target_vals

full_ks torch.Size([1000, 60])


tensor([17, 57, 27, 18, 58])

In [42]:
full_ks[30][30:40]

tensor([ 0.1688, -0.0007, -0.1307, -0.0585,  0.1896,  0.0085,  0.0352,  0.1662,
        -0.1187,  0.0198])

In [43]:
sub_ks.shape 

torch.Size([10, 3, 20])

In [44]:
sub_ks[3,1,-10:]

tensor([ 0.2923, -0.0012, -0.2264, -0.1014,  0.3284,  0.0147,  0.0610,  0.2878,
        -0.2057,  0.0342])

In [45]:
full_ks.shape

torch.Size([1000, 60])

In [46]:
sub_ks.shape

torch.Size([10, 3, 20])

In [47]:
# chunks, batch, neurons
sims = torch.einsum('ncd,bcd->bcn', sub_ks, sub_qs)
sims.shape

torch.Size([128, 3, 10])

In [48]:
top_sub_sims, inds = torch.topk(sims, k, dim=-1)
inds.shape

torch.Size([128, 3, 5])

In [49]:
inds[0]

tensor([[0, 9, 1, 6, 8],
        [1, 5, 2, 3, 8],
        [7, 8, 1, 2, 4]])

In [50]:
og_index_mags

tensor([[[100.],
         [ 10.],
         [  1.]]])

In [51]:
target_vals

tensor([17, 57, 27, 18, 58])

In [52]:
sum_res = None
# TODO: make this faster. maybe using:
# I am doing the smae thing but broadcasting and can work with more keys though. there is another implementation that is even better somewhere? 

for i in range(nchunks-1):
    if i ==0:
        sum_res = (top_sub_sims[:, i, :].unsqueeze(2) + top_sub_sims[:, i+1, :].unsqueeze(1)).flatten(start_dim=1)
    else: 
        sum_res = (sum_res.unsqueeze(2) + top_sub_sims[:, i+1, :].unsqueeze(1)).flatten(start_dim=1)
sum_res.shape 

torch.Size([128, 125])

In [53]:
sum_vals, sum_inds = torch.topk(sum_res, k, dim=-1)
# need to map these indices back to their original combos. 
sum_inds.shape, sum_inds[0] 

(torch.Size([128, 5]), tensor([ 0,  5, 10,  1,  6]))

In [54]:
relative_inds = []
relative_index_mags = [(k**(nchunks -i-1)) for i in range(nchunks)]
for mag in relative_index_mags:
    amount = sum_inds //mag
    # remainder
    sum_inds = sum_inds - (amount*mag)
    relative_inds.append( amount )
og_inds = torch.gather(inds, -1, torch.stack(relative_inds,dim=1))
og_inds[0]

/Users/trentonbricken/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  after removing the cwd from sys.path.


tensor([[0, 0, 0, 0, 0],
        [1, 5, 2, 1, 5],
        [7, 7, 7, 8, 8]])

In [59]:
sum_inds.shape, sum_vals.shape 

[autoreload of py_scripts.utils_DL failed: Traceback (most recent call last):
  File "/Users/trentonbricken/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/trentonbricken/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/trentonbricken/anaconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/trentonbricken/anaconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/trentonbricken/Foundational-SDM/py_scripts/utils_DL.py", line 3, in <module>
    from py_scripts import KDecayOptions, M

(torch.Size([128, 5]), torch.Size([128, 5]))

In [55]:
target_vals

tensor([17, 57, 27, 18, 58])

In [56]:
og_index_mags = [(n_explicit_keys_per_chunk**(nchunks -i-1)) for i in range(nchunks)]
og_index_mags = torch.Tensor(og_index_mags).unsqueeze(0).unsqueeze(-1)

implicit_neuron_inds = (og_inds * og_index_mags).sum(1)

In [57]:
implicit_neuron_inds[0]

tensor([17., 57., 27., 18., 58.])

In [58]:
target_vals

tensor([17, 57, 27, 18, 58])